In [27]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
import numpy as np

train=pd.read_csv('data/train.tsv', sep='\t',header=0)
test=pd.read_csv('data/test.tsv', sep='\t',header=0)

In [28]:
X_train = train[['sentence']]
y_train = train.label
X_test = test[['sentence']]
y_test = test.label

In [29]:
import jieba
X_train['cutted_comment'] = X_train.sentence.apply(lambda x: " ".join(jieba.cut(x)))
X_test['cutted_comment'] = X_test.sentence.apply(lambda x: " ".join(jieba.cut(x)))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
X_test.cutted_comment[:5]

0    派思 股份 与 自贡 华燃 之间 的 交易 ， 便是 根据 辽宁 众华 出具 的 上述 评估...
1    市场 认为 ， 从 财务数据 来看 ， 宣亚 的 收购 是 划算 的 。 宣亚 2016 年...
2    对于 业绩 下滑 ， 智慧 松德 给出 的 解释 是 ， 承担 公司 主要 智能 装备 业务...
3    此前 ， 不少 投资者 在 股 吧 中 讨论 认为 ， 甲基 卡锂 矿复产 将 对 融捷 股...
4    问及 信 达 生物 此前 主动 撤回 信迪 单抗 注射液 上市 申请 是否 影响 其 港股 ...
Name: cutted_comment, dtype: object

In [31]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
stop_words_file = "stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from pprint import pprint
pipe = Pipeline([
    ('vect', CountVectorizer(token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                             max_df = 0.5,
                             min_df=3,
                             max_features = None,
                             ngram_range = (1, 1),
                             stop_words = frozenset(stopwords))),
    ('tfidf', TfidfTransformer(norm = 'l2', use_idf = 'True')),
    ('svm', SVC()),
])

In [36]:
parameters = [{'svm__kernel': ['rbf'], 'svm__gamma': [1e-3, 1e-4],'svm__C': [1, 10, 100, 1000]},
              {'svm__kernel': ['linear'], 'svm__C': [1, 3,5,7,10]}]
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipe, parameters, scoring='f1_macro', cv=5, n_jobs=-1, verbose=1)

In [37]:
grid_search.fit(X_train.cutted_comment, y_train)

Fitting 5 folds for each of 13 candidates, totalling 65 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   36.8s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:  1.1min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.5,
                                                        max_features=None,
                                                        min_df=3,
                                                        ngram_range=(1, 1),
                                            

In [38]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param in parameters:
    for param_name in sorted(param.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.677
Best parameters set:
	svm__C: 1
	svm__gamma: 'auto_deprecated'
	svm__kernel: 'linear'
	svm__C: 1
	svm__kernel: 'linear'


In [39]:
best_parameters['svm']

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [40]:
y_pre = grid_search.predict(X_test.cutted_comment)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pre))

              precision    recall  f1-score   support

           0       0.66      0.69      0.67       725
           1       0.76      0.57      0.65       368
           2       0.76      0.81      0.79       907

    accuracy                           0.72      2000
   macro avg       0.73      0.69      0.70      2000
weighted avg       0.72      0.72      0.72      2000



In [41]:
y_pre = grid_search.predict(X_train.cutted_comment)
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pre))


              precision    recall  f1-score   support

           0       0.86      0.85      0.85      2157
           1       0.91      0.79      0.85      1072
           2       0.89      0.93      0.91      2771

    accuracy                           0.88      6000
   macro avg       0.88      0.86      0.87      6000
weighted avg       0.88      0.88      0.88      6000

